# **Tech Challenge 2 - Previsão Ibovespa**

# **1. Importações**

In [ ]:
# Manipulação de dados
import polars as pl
import pandas as pd
import numpy as np

# Banco de dados
import sqlalchemy as db

# Visualização
import matplotlib.pyplot as plt

# Prophet
from prophet import Prophet

# Configurações
plt.style.use("seaborn-v0_8")

## **2. Carregamento dos Dados (CSV)**

Arquivo extraído do Investing.com com dados diários do Ibovespa.

Datas extraídas:
**01.01.2024 - 01.12.2025**

In [ ]:
path = "data/dados_ibovespa.csv"

df_raw = pd.read_csv(path)
df_raw.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,01.12.2025,158.611,159.073,159.224,158.029,"7,50B","-0,29%"
1,28.11.2025,159.072,158.358,159.689,158.078,"8,02B","0,45%"
2,27.11.2025,158.360,158.554,158.864,158.167,"4,47B","-0,12%"
3,26.11.2025,158.555,155.915,158.714,155.914,"8,60B","1,70%"
4,25.11.2025,155.910,155.278,156.373,154.821,"6,93B","0,41%"


# **3. Pré-Processamento dos Dados**
##3.1 Padronização dos nomes das colunas

O arquivo do Investing vem com acentos e espaços, o que precisa ser ajustado.

In [ ]:
df = df_raw.rename(columns={
    "Data": "date",
    "Último": "close",
    "Abertura": "open",
    "Máxima": "high",
    "Mínima": "low",
    "Vol.": "volume",
    "Var%": "variation"
})

## 3.2 Conversão de tipos


In [ ]:
# Converter datas
df["date"] = pd.to_datetime(
    df["date"],
    format="%d.%m.%Y"
)

# Colunas numéricas
price_cols = ["close", "open", "high", "low"]

for col in price_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

## **3.3 Volume**

Convertendo a coluna e tratando valores nulos

In [ ]:
def parse_volume(v):
    """
    Converte a coluna de volume para formato numérico absoluto.

    O volume pode vir em diferentes escalas:
    - K: milhares
    - M: milhões
    - B: bilhões

    Também trata valores ausentes ou inválidos.
    """

    # Verifica valores nulos, vazios ou representados por "-"
    if pd.isna(v) or v in ["-", ""]:
        return np.nan

    # Converte para string, remove espaços extras e padroniza para maiúsculas
    v = str(v).strip().upper()

    # Converte vírgula decimal para ponto
    v = v.replace(".", "").replace(",", ".")

    # Se o volume estiver em bilhões (B)
    if v.endswith("B"):
        return float(v.replace("B", "")) * 1_000_000_000

    # Se o volume estiver em milhões (M)
    if v.endswith("M"):
        return float(v.replace("M", "")) * 1_000_000

    # Se o volume estiver em milhares (K)
    if v.endswith("K"):
        return float(v.replace("K", "")) * 1_000

    # Caso o valor já esteja em formato numérico simples
    return float(v)


## **3.4 Variação**

Tratando coluna Variação

In [ ]:
df["variation"] = (
    df["variation"]
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

## **4. Ordenação Temporal e Limpeza**

Removendo dias faltantes (Ibovespa não funciona nos finais de semana e feriados)


In [ ]:
df = df.sort_values("date").reset_index(drop=True)
df = df.dropna(subset=["close"])

df.head()

,date,close,open,high,low,volume,variation
0,2024-01-02,132697.0,134186.0,134195.0,132095.0,"8,44M",-1.11
1,2024-01-03,132834.0,132697.0,133576.0,13225.0,"8,70M",0.10
2,2024-01-04,131226.0,132831.0,132885.0,131024.0,"8,97M",-1.21
3,2024-01-05,132023.0,131218.0,132635.0,130579.0,"9,20M",0.61
4,2024-01-08,132427.0,132023.0,132498.0,131015.0,"8,50M",0.31
